### Phecodes consolidated
Putting ICD9- and ICD10- mapped phecodes into one csv file. This version maps phecodes to corresponding phenotype from ICD-10 mapped phecodes where applicable.

In [ ]:
import pandas as pd
import os

In [ ]:
# Read in phecode files
icd9 = pd.read_csv("phecode_icd9_rolled.csv",  encoding='latin-1')
icd10 = pd.read_csv("Phecode_map_v1_2_icd10cm_beta.csv",  encoding='latin-1')

In [ ]:
len(icd10['phecode'].unique())

In [ ]:
icd9[icd9['ICD9 String'] == 'Cholera']

In [ ]:
icd10[icd10['icd10cm_str'] == 'Cholera']

In [ ]:
# Remove excess columns
icd9 = icd9.drop(['Rollup', 'Leaf', 'Ignore Bool'], axis=1)
icd10 = icd10.drop(['leaf', 'rollup'], axis=1)

In [ ]:
# Change column names so that they are the same in the two dataframes:
# icd
# icd_string
# phecode
# phenotype
# excl_phecodes
# excl_phenotypes

In [ ]:
icd9 = icd9.rename({'ICD9' : 'icd',
                    'ICD9 String' : 'icd_string',
                    'PheCode' : 'phecode',
                    'Phenotype' : 'phenotype',
                    'Excl. Phecodes' : 'excl_phecodes',
                    'Excl. Phenotypes' : 'excl_phenotypes'}, axis=1)
    
icd10 = icd10.rename({'icd10cm' : 'icd',
                      'icd10cm_str' : 'icd_string',
                      'phecode_str' : 'phenotype',
                      'exclude_range' : 'excl_phecodes',
                      'exclude_name' : 'excl_phenotypes'}, axis=1)

In [ ]:
# Check same columns
icd9.columns == icd10.columns

In [ ]:
len(icd9['phecode'].unique())

In [ ]:
len(icd10['phecode'].unique())

#### Map ICD-9-mapped phecodes to corresponding phenotype for same ICD-10-mapped phecodes where applicable

In [ ]:
# ICD-9-mapped phecodes also in ICD-10-mapped phecodes
icd9_b = icd9[icd9['phecode'].isin(icd10['phecode'])]

In [ ]:
icd9_b.shape

In [ ]:
icd9_b[['phecode', 'phenotype']].drop_duplicates().shape

In [ ]:
# ICD-9-mapped phecodes that are NOT in ICD-10-mapped phecodes
# This will be left alone and eventually concatenated with icd9_b
# after corresponding phenotypes are mapped for that dataframe
icd9_only = icd9[~icd9['phecode'].isin(icd10['phecode'])]

In [ ]:
# Only keep first three columns for icd9_b
icd9_b = icd9_b[['icd', 'icd_string', 'phecode']]

In [ ]:
# Merge ICD-10 phenotype, excl_phecodes, excl_phenotypes
icd9_b = icd9_b.merge(icd10[['phecode', 'phenotype', 'excl_phecodes', 'excl_phenotypes']],
                      how='inner',
                      on='phecode').drop_duplicates()

In [ ]:
icd9_b = icd9_b.reset_index().drop(['index'], axis=1)

In [ ]:
# Concatenate icd9_b and icd9_only, replacing icd9
icd9 = pd.concat([icd9_b, icd9_only])

In [ ]:
# Concatenate
icd = pd.concat([icd9, icd10])

In [ ]:
icd.shape

In [ ]:
# Check that concatenation worked
(icd9.shape[0] + icd10.shape[0]) == icd.shape[0]

Check only one phenotype per phecode

In [ ]:
icd['phecode'].drop_duplicates().shape

In [ ]:
icd['phenotype'].drop_duplicates().shape

In [ ]:
icd[['phecode', 'phenotype']].drop_duplicates().shape

In [ ]:
# Save
# Database name: phecodes_v2
#icd.to_csv('phecodes/all_phecodes_v2.csv', index=False)

In [ ]:
pd.read_csv('all_phecodes_v2.csv')

#### Make modified icd dataframe with excl_phenotypes corresponding to values in the icd10 dataframe


In [ ]:
# Excluded phenotypes in icd10 phecodes roughly correspond to ICD10 categories
icd10['excl_phenotypes'].value_counts()

In [ ]:
# Excluded phenotypes in icd dataframe
icd['excl_phenotypes'].value_counts()

In [ ]:
icd.columns

In [ ]:
icd_trunc = icd[['icd', 'icd_string', 'phecode', 'phenotype', 'excl_phecodes']]

In [ ]:
# Need to drop duplicates because otherwise it'll be one row per phecode value found 
# in icd10. So for example, there are 30 rows of phecode == 8.0 in icd10 df, so this
# phecode will show up 30x in the df below if duplicates are not dropped.
icd_trunc = icd_trunc.merge(right=icd10[['phecode', 'excl_phenotypes']],
                            on='phecode').drop_duplicates()

In [ ]:
icd_trunc.shape

In [ ]:
icd.shape

In [ ]:
# Explore which phecodes are not in icd10 df
icd[~icd['phecode'].isin(icd_trunc['phecode'])]

In [ ]:
icd[~icd['phecode'].isin(icd_trunc['phecode'])]['excl_phenotypes'].value_counts().index.to_list()

In [ ]:
icd_trunc['excl_phenotypes'].value_counts()

In [ ]:
len(icd_trunc['excl_phenotypes'].value_counts())

In [ ]:
# Save
# Database name: phecodes_cat_v2
icd_trunc.to_csv('cat_phecodes_v2.csv', index=False)

In [ ]:
len(icd_trunc['phecode'].unique())